In [1]:
# Setup environment

# standard libraries
import numpy as np                        # library to handle data in a vectorized manner
import pandas as pd                       # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Foursquare
!conda install -c conda-forge geopy --yes  # find latitude/longitude of a place by name
from geopy.geocoders import Nominatim      # convert an address into latitude and longitude values

# JSON files
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

# plotting data
import matplotlib.cm as cm
import matplotlib.colors as colors

# k-means clustering
from sklearn.cluster import KMeans

# map display
!conda install -c conda-forge folium=0.5.0 --yes
import folium                             # map rendering library

# other functions
!pip install lxml                         # for read_html
!pip install beautifulsoup4               # requested by read_html
import itertools

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


In [2]:
# get list of major cities in California
# source: Wikipedia "List of largest California Cities"

lCities = pd.read_html('https://en.wikipedia.org/wiki/List_of_largest_California_cities_by_population')
dfCities = pd.DataFrame(lCities[0])    # add [0] to capture table properly
dfCities.head(10)                      # view first rows to verify dataframe looks correct

,2018 Rank[1],City,Population(2018 est.)[1],County,Image,Description
0,1,Los Angeles,3990456,Los Angeles,NaN,Los Angeles is one of the world's centers of m...
1,2,San Diego,1425976,San Diego,NaN,San Diego sits in the extreme south of Califor...
2,3,San Jose,1030119,Santa Clara,NaN,"San Jose was founded on November 29, 1777, as ..."
3,4,San Francisco,883305,San Francisco,NaN,San Francisco is a popular international touri...
4,5,Fresno,530093,Fresno,NaN,Fresno is at the heart of the San Joaquin Vall...
5,6,Sacramento,508529,Sacramento,NaN,Sacramento has been the state capital of Calif...
6,7,Long Beach,467354,Los Angeles,NaN,Long Beach is a coastal city in the Greater Lo...
7,8,Oakland,429082,Alameda,NaN,Oakland is a major West Coast port and is home...
8,9,Bakersfield,383579,Kern,NaN,Bakersfield sits at the southern end of the Sa...
9,10,Anaheim,352005,Orange,NaN,Anaheim is the most populous city in Orange Co...


In [3]:
# foursquare credentials

CLIENT_ID = 'XO2ABQ5J2EXXKQJTQNMEHEQVMSPBRE1AWOWMD0AOVDL2YG1S'     # your Foursquare ID
CLIENT_SECRET = '22IN2LGH2IWXTMTKDEU4PQRAIEVOEM2V5HMVFC2WI5IWNLRZ' # your Foursquare Secret
VERSION = '20180605'                                               # Foursquare API version

print('Foursquare credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Foursquare credentails:
CLIENT_ID: XO2ABQ5J2EXXKQJTQNMEHEQVMSPBRE1AWOWMD0AOVDL2YG1S
CLIENT_SECRET:22IN2LGH2IWXTMTKDEU4PQRAIEVOEM2V5HMVFC2WI5IWNLRZ


In [4]:
# function that extracts the category of the venue
# from class example

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [5]:
# get data from foursquare

nMax = 100                            # set maximum number of responses
# note: for a venues/explore?near query, if a radius is not specified,
# the default is "city-wide," so do not specify a radius for this query

# use first (largest) city in California, Los Angeles
xCity = dfCities.loc[0, 'City'] + ', CA'
print(xCity)

# the foursquare categoryId for "EV Charging Station" is 5032872391d4c4b30a586d64 (from foursquare documentation)
xCharging = '5032872391d4c4b30a586d64'

# construct URL
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&near={}&categoryId={}&limit={}'.format(
CLIENT_ID, CLIENT_SECRET, VERSION, xCity, xCharging, nMax)
    
results = requests.get(url).json()
venues = results['response']['groups'][0]['items']
lStations = json_normalize(venues) # flatten JSON
filtered_columns = ['venue.id', 'venue.name', 'venue.location.lat', 'venue.location.lng', 'venue.location.city']
lStations = lStations.loc[:, filtered_columns]
print(lStations.head(10))

Los Angeles, CA
                   venue.id                              venue.name  \
0  5c71bdc3f709c1002c848541                      Tesla Supercharger   
1  5c8fc732610f04002c909a18                      Tesla Supercharger   
2  54fcf277498e38ba3d5ec200          Tesla Supercharger Culver City   
3  4f8b09d7e4b0e5ed75e8867b         EV Charging - LAX Parking Lot 6   
4  51782415498ee6e7b8705246             Tabasco Auto Rapair & Tires   
5  50886290e4b0b52a6ae2ff9b                       pelican parts inc   
6  5a8f2feb6c08d17d5804a375  Northridge Library EV Charging Station   
7  58d32521761b1a449bbbf466    Electrical Charging Station (La Zoo)   
8  50492ba4e4b0baed1c3c3b5b               Hertz EV Charging Station   
9  5ac81b06535d6f631c5c0d28    Ev Charging - Westfield Century City   

   venue.location.lat  venue.location.lng venue.location.city  
0           34.014690         -118.493792        Santa Monica  
1           34.047386         -118.264624         Los Angeles  
2         

In [6]:
# create custom lists of venues in each of several amenity classes:
# restaurants, entertainment, lodging, leisurely shopping, services, 
# links to other forms of transportation
# note: I chose to populate dataframe manually in this notebook in order
# to remove need for a supporting external file

# create tuples of venue categories in category classes, such as
# "long duration shopping" or "entertainment"
# created manually from foursquare.com list of possible categories
# note: I selected these categories as potentially compatible activities
# for concurrent EV charging, based on the idea that these activities would
# typically take over 30 minutes and generally an hour or two, allowing
# sufficient time for at least a partial EV charge
# ideas for improvement: justify categories in each list using average "wait times" 
# from source such as Google Maps or a wait time study, or provide an interface for
# the user to select categories of interest from a list

lRestaurants = [('Food', '4d4b7105d754a06374d81259'),
               ('Street Food Gathering', '53e0feef498e5aac066fd8a9')]
tRestaurants = ['Restaurants'] * len(lRestaurants)
lEntertainment = [('Arts & Entertainment', '4d4b7104d754a06370d81259'),
                  ('Boat Rental', '5744ccdfe4b0c0459246b4c1'),
                  ('College Stadium', '4bf58dd8d48988d1b4941735'),
                  ('College Theater', '4bf58dd8d48988d1ac941735'),
                  ('Event', '4d4b7105d754a06373d81259'),
                  ('Night Market', '53e510b7498ebcb1801b55d4'),
                  ('Outdoors & Recreation', '4d4b7105d754a06377d81259'),
                  ('Community Center', '52e81612bcbc57f1066b7a34'),
                  ('Cultural Center', '52e81612bcbc57f1066b7a32')]
tEntertainment = ['Entertainment'] * len(lEntertainment)
lLodging = [('Hotel', '4bf58dd8d48988d1fa931735')]
tLodging = ['Lodging'] * len(lLodging)
lShopping = [('Big Box Store', '52f2ab2ebcbc57f1066b8b42'),
             ('Farmers Market', '4bf58dd8d48988d1fa941735'),
             ('Grocery Store', '4bf58dd8d48988d118951735'),
             ('Market', '50be8ee891d4fa8dcc7199a7'),
             ('Organic Grocery', '52f2ab2ebcbc57f1066b8b45'),
             ('Outlet Mall', '5744ccdfe4b0c0459246b4df'),
             ('Shopping Mall', '4bf58dd8d48988d1fd941735'),
             ('Shopping Plaza', '5744ccdfe4b0c0459246b4dc'),
             ('Supermarket', '52f2ab2ebcbc57f1066b8b46'),
             ('Warehouse Store', '52e816a6bcbc57f1066b7a54')]
tShopping = ['Long Stop Shopping'] * len(lShopping)
lServices = [('Library', '4bf58dd8d48988d12f941735'),
             ('Medical Center', '4bf58dd8d48988d104941735'),
             ('Internet Cafe', '4bf58dd8d48988d1f0941735')]
tServices = ['Long Stop Services'] * len(lServices)
lTransportation = [('Boat or Ferry', '4bf58dd8d48988d12d951735'),
                   ('Bus Station', '4bf58dd8d48988d1fe931735'),
                   ('Light Rail Station', '4bf58dd8d48988d1fc931735'),
                   ('Metro Station', '4bf58dd8d48988d1fd931735'),
                   ('Train Station', '4bf58dd8d48988d129951735'),
                   ('Tram Station', '52f2ab2ebcbc57f1066b8b51')]
tTransportation = ['Transportation Park-Ride Links'] * len(lTransportation)

# merge class lists into dfClasses dataframe
dfClassR = pd.DataFrame(tRestaurants, columns=['class'])
dfClassE = pd.DataFrame(tEntertainment, columns=['class'])
dfClassL = pd.DataFrame(tLodging, columns=['class'])
dfClassS = pd.DataFrame(tShopping, columns=['class'])
dfClassV = pd.DataFrame(tServices, columns=['class'])
dfClassT = pd.DataFrame(tTransportation, columns=['class'])
dfClasses = pd.concat([dfClassR, dfClassE, dfClassL, dfClassS,
                      dfClassV, dfClassT], ignore_index=True)

dfCatR = pd.DataFrame(lRestaurants, columns=['cat_name', 'cat_id'])
dfCatE = pd.DataFrame(lEntertainment, columns=['cat_name', 'cat_id'])
dfCatL = pd.DataFrame(lLodging, columns=['cat_name', 'cat_id'])
dfCatS = pd.DataFrame(lShopping, columns=['cat_name', 'cat_id'])
dfCatV = pd.DataFrame(lServices, columns=['cat_name', 'cat_id'])
dfCatT = pd.DataFrame(lTransportation, columns=['cat_name', 'cat_id'])
dfCategories = pd.concat([dfCatR, dfCatE, dfCatL, dfCatS, dfCatV, dfCatT], ignore_index=True)

dfClassList = pd.concat([dfClasses, dfCategories], axis=1, ignore_index=False)
print(dfClassList)


                             class               cat_name  \
0                      Restaurants                   Food   
1                      Restaurants  Street Food Gathering   
2                    Entertainment   Arts & Entertainment   
3                    Entertainment            Boat Rental   
4                    Entertainment        College Stadium   
5                    Entertainment        College Theater   
6                    Entertainment                  Event   
7                    Entertainment           Night Market   
8                    Entertainment  Outdoors & Recreation   
9                    Entertainment       Community Center   
10                   Entertainment        Cultural Center   
11                         Lodging                  Hotel   
12              Long Stop Shopping          Big Box Store   
13              Long Stop Shopping         Farmers Market   
14              Long Stop Shopping          Grocery Store   
15              Long Sto

In [18]:
dfClassList

,class,cat_name,cat_id
0,Restaurants,Food,4d4b7105d754a06374d81259
1,Restaurants,Street Food Gathering,53e0feef498e5aac066fd8a9
2,Entertainment,Arts & Entertainment,4d4b7104d754a06370d81259
3,Entertainment,Boat Rental,5744ccdfe4b0c0459246b4c1
4,Entertainment,College Stadium,4bf58dd8d48988d1b4941735
5,Entertainment,College Theater,4bf58dd8d48988d1ac941735
6,Entertainment,Event,4d4b7105d754a06373d81259
7,Entertainment,Night Market,53e510b7498ebcb1801b55d4
8,Entertainment,Outdoors & Recreation,4d4b7105d754a06377d81259
9,Entertainment,Community Center,52e81612bcbc57f1066b7a34


In [7]:
# create list of venue category id's for use in foursquare.com query
# no quotes, no square brackets, comma separated, no extra spaces

strClassList = ','.join(dfClassList['cat_id'].values)
print(strClassList)


4d4b7105d754a06374d81259,53e0feef498e5aac066fd8a9,4d4b7104d754a06370d81259,5744ccdfe4b0c0459246b4c1,4bf58dd8d48988d1b4941735,4bf58dd8d48988d1ac941735,4d4b7105d754a06373d81259,53e510b7498ebcb1801b55d4,4d4b7105d754a06377d81259,52e81612bcbc57f1066b7a34,52e81612bcbc57f1066b7a32,4bf58dd8d48988d1fa931735,52f2ab2ebcbc57f1066b8b42,4bf58dd8d48988d1fa941735,4bf58dd8d48988d118951735,50be8ee891d4fa8dcc7199a7,52f2ab2ebcbc57f1066b8b45,5744ccdfe4b0c0459246b4df,4bf58dd8d48988d1fd941735,5744ccdfe4b0c0459246b4dc,52f2ab2ebcbc57f1066b8b46,52e816a6bcbc57f1066b7a54,4bf58dd8d48988d12f941735,4bf58dd8d48988d104941735,4bf58dd8d48988d1f0941735,4bf58dd8d48988d12d951735,4bf58dd8d48988d1fe931735,4bf58dd8d48988d1fc931735,4bf58dd8d48988d1fd931735,4bf58dd8d48988d129951735,52f2ab2ebcbc57f1066b8b51


In [8]:
# build dataframe of stations and their amenities
# stationID: foursquare's unique ID for each charging station
# amenityID: foursquare's unique ID for each amenity (venue) nearby the charging station
# amenityName: name of amenity 
# amenityLat: latitude of amenity
# amenityLong: longitude of amenity
# amenityCity: city of amenity
# amenityCatID: primary category ID of amenity
# amenityCatName: primary category name of amenity

# intialize dataframe to hold stations & venues list & setup variables
dfAmenities = pd.DataFrame()
dfSummary = pd.DataFrame(columns=['stationID', 'countAmenities'])
dfSummary['stationID'] = lStations['venue.id']
nStations = len(lStations['venue.id'])
# note: also use strClassList, defined earlier
nRadius = 400     # radius to other venues: 400 m is approx 1/4 mile
                  # reference: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3377942/
                  # reference: https://www.smartcitiesdive.com/ex/sustainablecitiescollective/pedestrians-and-park-planning-how-far-will-people-walk/24937/
nMax = 50         # maximum number of venues to retreive

print("# of stations: " + str(nStations))

# retrieve nearby venues to each station in lStations
# also create a list of # of nearby amentities per station
for iStation in range(nStations):
    xLat = lStations.loc[iStation, 'venue.location.lat']
    xLong = lStations.loc[iStation, 'venue.location.lng']
    
    print("Station # " + str(iStation) + ": ")

    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
           CLIENT_ID, CLIENT_SECRET, VERSION, xLat, xLong, strClassList, nRadius, nMax)
        
    results = requests.get(url).json()
    venues = results['response']['groups'][0]['items']
    lAmenities = json_normalize(venues)                  # flatten JSON
    filtered_columns = ['venue.id', 'venue.name', 'venue.location.lat', 
                        'venue.location.lng', 'venue.location.city', 'venue.categories']
    lAmenities = lAmenities.loc[:, filtered_columns]
    nAmenities = lAmenities.shape[0]
    dfSummary.loc[iStation, 'countAmenities'] = nAmenities

    # make list of venue category id's & venue category name's for each venue in lAmenities
    print('# of nearby amenities: ' + str(nAmenities))
    dfCategories = pd.DataFrame(columns=['catID', 'catName'])
    for iAmenity in range(nAmenities):
        dfCategories.loc[iAmenity] = [lAmenities['venue.categories'][iAmenity][0].get('id'), 
                             lAmenities['venue.categories'][iAmenity][0].get('name')]
        # end for loop
    dfAmenities = dfAmenities.append(list(zip(list(itertools.repeat(lStations['venue.id'][iStation],
                                     nAmenities)), lAmenities['venue.id'],
                                     lAmenities['venue.name'], lAmenities['venue.location.lat'],
                                     lAmenities['venue.location.lng'], lAmenities['venue.location.city'],
                                     dfCategories['catID'].values,
                                     dfCategories['catName'].values)),
                                    ignore_index=True)
# end for loop

dfAmenities.rename(columns={0:'stationID', 1: 'amenityID', 2: 'amenityName', 3: 'amenityLat',
                           4: 'amenityLong', 5: 'amenityCity', 6: 'amenityCatID', 7: 'amenityCatName'}, inplace=True)
print(dfAmenities.head(10))


# of stations: 16
Station # 0: 
# of nearby amenities: 50
Station # 1: 
# of nearby amenities: 46
Station # 2: 
# of nearby amenities: 33
Station # 3: 
# of nearby amenities: 50
Station # 4: 
# of nearby amenities: 6
Station # 5: 
# of nearby amenities: 6
Station # 6: 
# of nearby amenities: 33
Station # 7: 
# of nearby amenities: 5
Station # 8: 
# of nearby amenities: 4
Station # 9: 
# of nearby amenities: 42
Station # 10: 
# of nearby amenities: 28
Station # 11: 
# of nearby amenities: 12
Station # 12: 
# of nearby amenities: 13
Station # 13: 
# of nearby amenities: 9
Station # 14: 
# of nearby amenities: 2
Station # 15: 
# of nearby amenities: 27
                  stationID                 amenityID  \
0  5c71bdc3f709c1002c848541  4f7f35020039ef8a139bd3aa   
1  5c71bdc3f709c1002c848541  562a6755498e0e92cb0d2d8a   
2  5c71bdc3f709c1002c848541  4dc1522745dd472e9b746f6b   
3  5c71bdc3f709c1002c848541  591df7df6fa81f3c8c1f4296   
4  5c71bdc3f709c1002c848541  59c16d483d4791079750dc97   


In [9]:
print(np.std(dfSummary['countAmenities']))
print(np.mean(dfSummary['countAmenities']))
dfSummary

17.156904586783714
22.875


,stationID,countAmenities
0,5c71bdc3f709c1002c848541,50
1,5c8fc732610f04002c909a18,46
2,54fcf277498e38ba3d5ec200,33
3,4f8b09d7e4b0e5ed75e8867b,50
4,51782415498ee6e7b8705246,6
5,50886290e4b0b52a6ae2ff9b,6
6,5a8f2feb6c08d17d5804a375,33
7,58d32521761b1a449bbbf466,5
8,50492ba4e4b0baed1c3c3b5b,4
9,5ac81b06535d6f631c5c0d28,42


In [10]:
# one hot encoding

dfAmenities = pd.concat([dfAmenities, pd.get_dummies(dfAmenities['amenityCatName'], 
              prefix='cat', dummy_na=True, prefix_sep='_')], axis=1)
print(dfAmenities.head(20))

# make list of column names & # column names available for use later
lCols = list(dfAmenities.columns)
nCols = len(list(dfAmenities.columns))
print('# columns in dfAmenities: ' + str(nCols))


                   stationID                 amenityID  \
0   5c71bdc3f709c1002c848541  4f7f35020039ef8a139bd3aa   
1   5c71bdc3f709c1002c848541  562a6755498e0e92cb0d2d8a   
2   5c71bdc3f709c1002c848541  4dc1522745dd472e9b746f6b   
3   5c71bdc3f709c1002c848541  591df7df6fa81f3c8c1f4296   
4   5c71bdc3f709c1002c848541  59c16d483d4791079750dc97   
5   5c71bdc3f709c1002c848541  56689cc0498e5a0ee52b4549   
6   5c71bdc3f709c1002c848541  5927a8fdcad1b6573021f649   
7   5c71bdc3f709c1002c848541  4da4bb1d0f578cfaafdbd518   
8   5c71bdc3f709c1002c848541  54cc4170498e6e5d8a8b0ea1   
9   5c71bdc3f709c1002c848541  55ce2ff5498e0153c737c871   
10  5c71bdc3f709c1002c848541  4bad0a75f964a52081273be3   
11  5c71bdc3f709c1002c848541  4a4ad4a2f964a52039ac1fe3   
12  5c71bdc3f709c1002c848541  4c58c69cf346c928902a4bca   
13  5c71bdc3f709c1002c848541  59404659e679bc120c643d3c   
14  5c71bdc3f709c1002c848541  4d3a0739979ea1431ac88bfc   
15  5c71bdc3f709c1002c848541  597af741acc5f5228a951df9   
16  5c71bdc3f7

In [11]:
# prepare for classification of stations by venue categories

# create dataframe which only contains stationID & one-hot encoded amenity categories
# to be used in classification
# note: due to the way the dataframe was constructed, it is already sorted by stationID

dfSubset = dfAmenities.copy(deep=False)
lCols_encoded = [0] + list(np.arange(8, nCols))
dfTestClassify = dfSubset[dfSubset.columns[lCols_encoded]]

# prepare for classification
dfTestClassify_grouped = dfTestClassify.groupby('stationID').mean().reset_index()
dfTestClassify_noID = dfTestClassify.drop('stationID', 1)


In [12]:
# k-means clustering

kClusters = 5            # arbitrarily choose # of clusters
kmStations = KMeans(n_clusters=kClusters, random_state=0).fit(dfTestClassify_noID)
lGroups = kmStations.labels_
dfGroups = pd.DataFrame(zip(dfTestClassify_grouped['stationID'], lGroups), columns=['stationID','Group'])
print(dfGroups.head(20))


                   stationID  Group
0   4f8b09d7e4b0e5ed75e8867b      4
1   50492ba4e4b0baed1c3c3b5b      1
2   50886290e4b0b52a6ae2ff9b      1
3   51782415498ee6e7b8705246      0
4   5179f614e4b027b155df66c5      1
5   526a0069498e71b4559f29ee      0
6   5420ddd9498e989575d71b79      0
7   54fcf277498e38ba3d5ec200      1
8   583a3de194c6904b8009c1f4      0
9   58d32521761b1a449bbbf466      0
10  5a8f2feb6c08d17d5804a375      0
11  5ac81b06535d6f631c5c0d28      0
12  5c71bdc3f709c1002c848541      0
13  5c8fc732610f04002c909a18      0
14  5dccf2c6f94e180008a4edee      0
15  5dd5b82c1d1198000877f2a2      0


In [13]:
# add the group # assigned by k-means clustering analysis to the list of stations & amenity categories
dfTestClassify_merged = dfTestClassify_grouped.join(dfGroups.set_index('stationID'), on='stationID')
print(dfTestClassify_merged.head(20))

# add lat/long information per stationID
dfLocations = pd.DataFrame(zip(lStations['venue.id'],
                               lStations['venue.name'],
                               lStations['venue.location.lat'],
                               lStations['venue.location.lng']), 
                           columns=['stationID', 'stationName', 'stationLat', 'stationLong'])
dfClassify_locations = dfTestClassify_merged.join(dfLocations.set_index('stationID'), on='stationID')
print(dfClassify_locations.head(20))


                   stationID  cat_American Restaurant  cat_Asian Restaurant  \
0   4f8b09d7e4b0e5ed75e8867b                 0.180000              0.020000   
1   50492ba4e4b0baed1c3c3b5b                 0.000000              0.000000   
2   50886290e4b0b52a6ae2ff9b                 0.000000              0.000000   
3   51782415498ee6e7b8705246                 0.000000              0.000000   
4   5179f614e4b027b155df66c5                 0.000000              0.000000   
5   526a0069498e71b4559f29ee                 0.222222              0.000000   
6   5420ddd9498e989575d71b79                 0.000000              0.000000   
7   54fcf277498e38ba3d5ec200                 0.000000              0.030303   
8   583a3de194c6904b8009c1f4                 0.000000              0.000000   
9   58d32521761b1a449bbbf466                 0.200000              0.000000   
10  5a8f2feb6c08d17d5804a375                 0.060606              0.060606   
11  5ac81b06535d6f631c5c0d28                 0.00000

In [17]:
dfClassify_locations.head(5)

,stationID,cat_American Restaurant,cat_Asian Restaurant,cat_Australian Restaurant,cat_BBQ Joint,cat_Bakery,cat_Bistro,cat_Brazilian Restaurant,cat_Breakfast Spot,cat_Burger Joint,cat_Burrito Place,cat_Cafeteria,cat_Café,cat_Chinese Restaurant,cat_Deli / Bodega,cat_Diner,cat_Donut Shop,cat_Falafel Restaurant,cat_Fast Food Restaurant,cat_Food,cat_Food Court,cat_Food Stand,cat_Food Truck,cat_French Restaurant,cat_Fried Chicken Joint,cat_Gastropub,cat_Greek Restaurant,cat_Hawaiian Restaurant,cat_Hot Dog Joint,cat_Indian Restaurant,cat_Italian Restaurant,cat_Japanese Restaurant,cat_Korean Restaurant,cat_Latin American Restaurant,cat_Lebanese Restaurant,cat_Mediterranean Restaurant,cat_Mexican Restaurant,cat_Middle Eastern Restaurant,cat_Mongolian Restaurant,cat_New American Restaurant,cat_Noodle House,cat_Persian Restaurant,cat_Pizza Place,cat_Poke Place,cat_Ramen Restaurant,cat_Restaurant,cat_Salad Place,cat_Sandwich Place,cat_Seafood Restaurant,cat_Snack Place,cat_Southern / Soul Food Restaurant,cat_Steakhouse,cat_Sushi Restaurant,cat_Szechuan Restaurant,cat_Taco Place,cat_Tex-Mex Restaurant,cat_Thai Restaurant,cat_Vegetarian / Vegan Restaurant,cat_Vietnamese Restaurant,cat_Wings Joint,cat_nan,Group,stationName,stationLat,stationLong
0,4f8b09d7e4b0e5ed75e8867b,0.18,0.02,0.0,0.0,0.020000,0.02,0.0,0.0,0.06,0.02,0.02,0.020000,0.00,0.02,0.0,0.020000,0.0,0.040000,0.040000,0.0,0.0,0.000000,0.000000,0.0,0.02,0.00,0.0,0.02,0.0,0.040000,0.0,0.0,0.0,0.0,0.02,0.060000,0.0,0.0,0.100000,0.0,0.0,0.100000,0.0,0.0,0.020000,0.0,0.040000,0.02,0.04,0.0,0.000000,0.0,0.0,0.02,0.0,0.000000,0.02,0.0,0.0,0.0,4,EV Charging - LAX Parking Lot 6,33.942957,-118.403722
1,50492ba4e4b0baed1c3c3b5b,0.00,0.00,0.0,0.0,0.000000,0.00,0.0,0.0,0.00,0.00,0.00,0.250000,0.25,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00,0.25,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.00,0.00,0.0,0.000000,0.0,0.0,0.25,0.0,0.000000,0.00,0.0,0.0,0.0,1,Hertz EV Charging Station,33.953742,-118.384432
2,50886290e4b0b52a6ae2ff9b,0.00,0.00,0.0,0.0,0.000000,0.00,0.0,0.0,0.00,0.00,0.00,0.000000,0.00,0.00,0.0,0.000000,0.0,0.000000,0.166667,0.0,0.0,0.500000,0.000000,0.0,0.00,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.00,0.00,0.0,0.166667,0.0,0.0,0.00,0.0,0.166667,0.00,0.0,0.0,0.0,1,pelican parts inc,33.807146,-118.305187
3,51782415498ee6e7b8705246,0.00,0.00,0.0,0.0,0.000000,0.00,0.0,0.0,0.00,0.00,0.00,0.000000,0.00,0.00,0.0,0.166667,0.0,0.166667,0.166667,0.0,0.0,0.000000,0.000000,0.0,0.00,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.166667,0.0,0.0,0.000000,0.0,0.0,0.166667,0.0,0.0,0.000000,0.0,0.166667,0.00,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.00,0.0,0.0,0.0,0,Tabasco Auto Rapair & Tires,34.232497,-118.369322
4,5179f614e4b027b155df66c5,0.00,0.00,0.0,0.0,0.076923,0.00,0.0,0.0,0.00,0.00,0.00,0.153846,0.00,0.00,0.0,0.000000,0.0,0.000000,0.076923,0.0,0.0,0.076923,0.153846,0.0,0.00,0.00,0.0,0.00,0.0,0.153846,0.0,0.0,0.0,0.0,0.00,0.076923,0.0,0.0,0.076923,0.0,0.0,0.076923,0.0,0.0,0.076923,0.0,0.000000,0.00,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.00,0.0,0.0,0.0,1,Urban Radish Charging Station,34.035950,-118.231560


In [14]:
# count top amenity categories for each stationID

# define function
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:-2]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

# initialize variables
nTops = 5        # arbitrarily choose how many "top venues" (most common amenity categories) to show
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
lColumns = ['stationID']
for ind in np.arange(nTops):
    try:
        lColumns.append('{}{} Most Common Category'.format(ind+1, indicators[ind]))
    except:
        lColumns.append('{}th Most Common Category'.format(ind+1))

# count top # of items
dfTops = pd.DataFrame(columns=lColumns)
nCategories = dfTestClassify_grouped.shape[0]
# note: nStations already defined above; nStations = # charging stations
dfTops['stationID'] = dfTestClassify_grouped['stationID']
for iRow in np.arange(nStations):             # for each row
    dfTops.loc[iRow, 1:] = return_most_common_venues(dfTestClassify.iloc[iRow, :], nTops)

# add group #'s to list of top categories
dfTops = dfTops.join(dfGroups.set_index('stationID'), on='stationID')
dfTops = dfTops.reindex(columns=['stationID','Group'] + list(dfTops.columns[1:-1]))

# add # amenities per stationID
dfTops = dfTops.join(dfSummary.set_index('stationID'), on='stationID')
print(dfTops)

                   stationID  Group 1st Most Common Category  \
0   4f8b09d7e4b0e5ed75e8867b      4   cat_Mexican Restaurant   
1   50492ba4e4b0baed1c3c3b5b      1  cat_American Restaurant   
2   50886290e4b0b52a6ae2ff9b      1  cat_American Restaurant   
3   51782415498ee6e7b8705246      0     cat_Sushi Restaurant   
4   5179f614e4b027b155df66c5      1  cat_American Restaurant   
5   526a0069498e71b4559f29ee      0  cat_Fried Chicken Joint   
6   5420ddd9498e989575d71b79      0   cat_Italian Restaurant   
7   54fcf277498e38ba3d5ec200      1  cat_American Restaurant   
8   583a3de194c6904b8009c1f4      0            cat_Gastropub   
9   58d32521761b1a449bbbf466      0          cat_Salad Place   
10  5a8f2feb6c08d17d5804a375      0         cat_Burger Joint   
11  5ac81b06535d6f631c5c0d28      0  cat_Japanese Restaurant   
12  5c71bdc3f709c1002c848541      0           cat_Food Court   
13  5c8fc732610f04002c909a18      0           cat_Steakhouse   
14  5dccf2c6f94e180008a4edee      0     

In [15]:
dfTops.head(5)

,stationID,Group,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category,countAmenities
0,4f8b09d7e4b0e5ed75e8867b,4,cat_Mexican Restaurant,cat_Vietnamese Restaurant,cat_Diner,cat_Greek Restaurant,cat_Gastropub,50
1,50492ba4e4b0baed1c3c3b5b,1,cat_American Restaurant,cat_Diner,cat_Hawaiian Restaurant,cat_Greek Restaurant,cat_Gastropub,4
2,50886290e4b0b52a6ae2ff9b,1,cat_American Restaurant,cat_Diner,cat_Hawaiian Restaurant,cat_Greek Restaurant,cat_Gastropub,6
3,51782415498ee6e7b8705246,0,cat_Sushi Restaurant,cat_Vietnamese Restaurant,cat_Deli / Bodega,cat_Greek Restaurant,cat_Gastropub,6
4,5179f614e4b027b155df66c5,1,cat_American Restaurant,cat_Diner,cat_Hawaiian Restaurant,cat_Greek Restaurant,cat_Gastropub,13


In [16]:
# map results of k-means clustering of charging stations on nearby amenities

# set map starting location
# note: use geopy to get lat/long of Los Angeles
mapAddress = 'Los Angeles, CA'
geolocator = Nominatim(user_agent="TO_explorer")
location = geolocator.geocode(mapAddress)
mapLat = location.latitude
mapLong = location.longitude

# create cluster map
map_clusters = folium.Map(location=[mapLat, mapLong], tiles='cartodbpositron', 
                          zoom_start=9)

# set color scheme for the clusters
x = np.arange(kClusters)
ys = [i + x + (i*x)**2 for i in range(kClusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, countAmenities in zip(dfClassify_locations['stationLat'], 
                                  dfClassify_locations['stationLong'], 
                                  dfClassify_locations['stationName'], 
                                  dfClassify_locations['Group'],
                                  dfTops['countAmenities']):
    label = folium.Popup(str(poi) + ' - Group # ' + str(cluster) + ' - ' + 
                         str(countAmenities) + ' nearby amenities', parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=2 + countAmenities / 2,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


<i>Note:</i> On the map, above, the size of the circular marker represents the number of nearby amenities.  Larger circles indicate more amenities within 1/4 mile of the charging station.  Colors indicate classification groups, based on types of nearby amenities.